# My Test (Word2Vec)

参考リンク https://qiita.com/makaishi2/items/63b7986f6da93dc55edd
まずは「三四郎」の冒頭部分を出しましょう。

In [4]:
import codecs
# ファイル読込み、内部表現化
f = codecs.open('sanshiro.txt', "r", "sjis")
text = f.read()
f.close()

In [11]:
# ファイル整形
import re
# ヘッダ部分の除去
# text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

In [13]:
# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


の評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。



次に Janome を使って見ましょう。

In [15]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

In [17]:
# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
for word in ret:
    print(word)

三四郎
京都
ちょっと
用
ある
降りる
ついで


In [19]:
# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

一
する
目
さめる
女
隣
じいさん
話
始める
いる


In [21]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [22]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['世間'])

[-0.65313166 -0.62891906  0.17438062  0.26170734  0.13503277  0.1434436
  0.3228045   0.16177198  0.2681557   0.20288964 -0.5050857  -0.27775013
 -0.6406938   0.26332816  0.65016234  0.16011722  0.39971462  0.21006517
  0.22237374  0.77312905 -0.08696599 -0.08956906 -0.75035876 -0.30150267
  0.02591058  0.75857884  0.5207765  -0.08462646  0.08198166 -0.79446936
 -0.36525548  0.24380173 -0.03011892 -0.70276207 -0.48627678  0.58974975
  0.51893663 -0.26492783  0.3536507   0.24789363 -0.43629184 -0.06122503
 -0.03264514  0.2280395   0.28010875  1.06901    -0.48890734 -0.1555945
  0.82734746 -0.00135017 -0.18063134 -0.13531289 -0.26069164  0.10685145
  0.78156453  0.46627632 -0.07300753 -0.35923982  0.00810226 -0.36684614
 -0.7153586   0.5566864  -0.09397646 -0.80985737 -0.32292515 -0.2210996
  0.7335055  -0.486771   -0.21325512  0.60944533  0.31590056 -0.09091179
  0.03709435  0.44146886  0.42885622 -0.47714093  0.6710077   0.04712195
  0.3305971  -0.24903543 -0.46706665  0.2170919  -0.17

In [23]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])

聞こえる 0.6189125776290894
自己 0.5277218222618103
堪える 0.5241377353668213
喝采 0.5240006446838379
酔う 0.5154030919075012
決心 0.5103549361228943
外国 0.5021237134933472
社会 0.486386775970459
未来 0.4771953225135803
賛成 0.46586984395980835


これが全て通ったらテストが完了です。